# JAGUAR E-TYPE DATA PULL

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import joblib
import os
import sys
import helpers

Get List of Every Result and Save:

In [11]:
#### Get List of Every Result and Save ####
#### DO NOT DELETE ####

make = input('Enter Car Make: ')
model = input('Enter Car Model: ')
open("./data/HTML/"+ make + model +".html", 'w').close()
ids, urls, t = helpers.getlistings(make, model)
data = ''
listings = []
for i in range(0,len(ids)):
    urlstart = 'https://bringatrailer.com/wp-json/bringatrailer/1.0/data/keyword-filter?bat_keyword_pages=' + ids[i] + '&sort=td&page='
    urlend = '&results=items'
    pgend = 15
    tempurl = helpers.geturls(urlstart, urlend, pgend, urls)
    listings = tempurl
    listings = listings[len(ids):len(listings)]

    def get_data(url):

        r = requests.get(url)

        soup = BeautifulSoup(r.content, "html.parser")
        return str(soup)


    souplist = joblib.Parallel(n_jobs=-2)(joblib.delayed(get_data)(url) for url in listings)


### WRITE DATA TO FILE ###
    delimiter = "%newline%"
    temp = delimiter.join(souplist)
    data = temp + delimiter + data
with open("./data/HTML/"+ make + model +".html", "a") as file:
    file.write(data)


display('done')

'done'

Pull Information Off Each Listing Stored in URLS List:

In [28]:
carlist = []
Car = []
### READ DATA ###

with open("./data/HTML/"+ make + model +".html", "r") as file:
    test_data = file.read()

out = test_data.split(delimiter)

for i in range(0,len(listings)):
    soup = BeautifulSoup(out[i], 'html.parser')
    title = str(soup.find('title'))
    if "Parts" not in title and "Tool" not in title and "Memorabilia" not in title and "Tool Kit" not in title and "Removable" not in title and "Gearbox" not in title and title != 'None' and 'listing' in listings[i]:
        loc = str(soup.findAll('a'))
        transmission = str(soup.findAll('li'))
        contents = str(soup.findAll('p'))
        essentials = soup.find('div', class_='essentials')

        #Get Car Description
        desc = helpers.getdesc(listings[i])


        #Get Year
        year = desc[0:4]
        
        #Check if Sold and get highest bid
        sold, price = helpers.getsaleprice(title, loc)


        #Get Transmission
        trans = 'Manual'
        if 'Automatic' in transmission:
            trans = 'Automatic'

        #Get Location

        town, state = helpers.getlocation(loc)

        
        #Check for Project
        if 'project' in title or 'Project' in title:
            project = 1
        else:
            project = 0
        
        #Get Lot Number
        try:
            index = title.index('Lot #')
            lotnumb = title[index+5:index+12].partition(')')[0]
        except:
            lotnumb = "N/A"
            
        #Get Month
        month = helpers.getmonth(title)
        
        #Get Year
        if '(' in title:
            index = title.index('(')
            yearsold = title[index-5:index-1]
        else:
            index = title.index('20')
            yearsold = title[index:index+5]
        
        #Get Engine and Number Matching
        numbmatch = 0
        
        if 'Numbers-Matching' in transmission:
            numbmatch = 1

        engine = helpers.getengine(transmission)
        
        #Get Various Indicators for rust, refurbishment, restoration, scratch, paint bubbles, metal repair, hardtop, or overdrive options
        rust, refurbished, restored, scratch, paintbub, metalrepair, hardtop, overdrive, turbo, super = helpers.getindicators(contents)
            
            
        #Get Mileage
        miles, milestmu = helpers.getmileage(transmission)

        #Get Descriptions of options and chassis #

        temp = helpers.getenginedesc(essentials, engine)
        chassis = temp[0]
        specialdesc = temp[1]
        mileagedesc = temp[2]
        enginedesc = temp[3]
        transdesc = temp[4]
        paintdesc = temp[5]
        interiordesc = temp[6]
        carbdesc = temp[7]
        wheeldesc = temp[8]
        brakedesc = temp[9]
        suspdesc = temp[10]
        engine = temp[11]

        #Check Trans with Description
        if 'Manual' in transdesc:
            trans = 'Manual'
        elif 'Automatic' in transdesc:
            trans = 'Automatic'

        #Check Mileage with Description
        try:
            if "K" in str(miles):
                index = mileagedesc.index("ilometers")
                tempmile = mileagedesc[0:index].partition(" ")[0]
                if "k" in tempmile or "K" in tempmile:
                    milestmu = re.sub(r"[^0-9]", "", tempmile)
                    milestmu = int(milestmu) * 1000
            elif "K" in str(milestmu):
                index = mileagedesc.index("ilometers")
                tempmile = mileagedesc[0:index].partition(" ")[0]
                if "k" in tempmile or "K" in tempmile:
                    miles = re.sub(r"[^0-9]", "", tempmile)
                    miles = int(miles) * 1000
        except:
            print('failed')
        #Check for bespoke bodies
        if 'Bertone' in title:
            bertone = 1
        else:
            bertone = 0
        if 'Zagato' in title:
            zagato = 1
        else:
            zagato = 0

        Car = [lotnumb, desc, trans, year, miles, milestmu, engine, turbo, super, numbmatch, hardtop, overdrive, scratch, paintbub, metalrepair, rust, refurbished, restored, project, sold, month, yearsold, price, town, state, zagato, bertone, chassis, specialdesc, mileagedesc, enginedesc, transdesc, paintdesc, interiordesc, carbdesc, wheeldesc, brakedesc, suspdesc, listings[i]]
        carlist.append(Car)
    else:
        print('Discarded')
    # index = transmission.index('aint')
    # print(transmission[index-20:index+20])

df = pd.DataFrame(carlist, columns=['Lot Number', 'Description', 'Transmission', 'Model Year', 'Mileage', 'Mileage (if TMU)','Engine', 'Turbocharged', 'Supercharged', 'Numbers Matching', 'Hardtop', 'Overdrive', 'Scratch', 'Paint Bubble', 'Metal Repair', 'Rust','Refurbished','Restored', 'Project', 'Sold', 'Month', 'Year', 'Price', 'City', 'State', 'Zagato', 'Bertone', 'Chassis', 'Special Description', 'Mileage Description', 'Engine Description', 'Transmission Description','Paint Description', 'Interior Description','Carburetor Description', 'Wheels Description', 'Brakes Description', 'Suspension Description', 'URL'])

display(df)



ValueError: substring not found

Save Data as Excel File:

In [10]:
path = 'data/' + make + '/'
try:
    os.mkdir(path)
except:
    print('Path already exists')
df.to_excel(path + make + '_' + model + '.xlsx')
print('Saved!')

Path already exists
Saved!


In [29]:
mileagedesc

'N/A'

In [21]:
helpers.getmileage(transmission)

('123000K', 'N/A')